In [1]:
### VIN DETECT & RECOGNISE ###
##VIN Detection - YOLO v5
##VIN Extraction
##Image Converion
##VIN recognition - PaddleOCR

In [1]:
def paddle_man(img_path):
    from paddleocr import PaddleOCR,draw_ocr
    from PIL import ImageFont
    import PIL
    ocr = PaddleOCR(det_model_dir='paddleOCR_V2_EN_GEN_SRV_MOB_T/ch_PP-OCRv2_det_distill_train', rec_model_dir='paddleOCR_V2_EN_GEN_SRV_MOB_T/ret_model/ch_PP-OCRv2_rec_train',cls_model_dir='paddleOCR_V2_EN_GEN_SRV_MOB_T/cls_model/ch_ppocr_mobile_v2.0_cls_train',use_angle_cls=True) 
    result = ocr.ocr(img_path,cls=True)
    txts = [line[1][0] for line in result]
    scores = [line[1][1] for line in result]
    
    # draw result
    from PIL import Image
    image = Image.open(img_path).convert('RGB')
    boxes = [line[0] for line in result]
    txts = [line[1][0] for line in result]
    scores = [line[1][1] for line in result]
    font = ImageFont.load_default()
    
    im_show = draw_ocr(image, boxes, txts, scores, font_path='L:/CV/PaddleOCR/PaddleOCR/doc/fonts/simfang.ttf')
    im_show = Image.fromarray(im_show)
    im_show.save('result.jpg')
    im = Image.open("result.jpg")
    im.show() 
    return txts,scores

def display(im_path):
    dpi = 80
    im_data = plt.imread(im_path)    
    height, width  = im_data.shape[:2]
    
    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)    
    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])    
    # Hide spines, ticks, etc.
    ax.axis('off')    
    # Display the image.
    ax.imshow(im_data, cmap='gray')    
    plt.show()

In [2]:
#https://stackoverflow.com/questions/28816046/
#displaying-different-images-with-actual-size-in-matplotlib-subplot
import cv2
from matplotlib import pyplot as plt
%matplotlib inline
def display(im_path):
    dpi = 80
    im_data = plt.imread(im_path)    
    height, width  = im_data.shape[:2]
    
    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)    
    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])    
    # Hide spines, ticks, etc.
    ax.axis('off')    
    # Display the image.
    ax.imshow(im_data, cmap='gray')    
    plt.show()

In [3]:
def main_code(img):
    ##Libraries
    import os
    import cv2
    import shutil
    
    #Remove previous results
    isdir = os.path.isdir('runs/detect')
    if isdir == True:
        shutil.rmtree(r'runs/detect')
    else:
        pass
    
    #Convert image to gray scale
    
    read_img = cv2.imread(img)
    gray_image=cv2.cvtColor(read_img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite('temp/gray_image.jpg', gray_image)
    
    #Detect VIN Region
    !python yolov5/detect.py --weights yolov5/yolov5/weights/large/best.pt --img 416 --conf 0.6 --source temp/gray_image.jpg --project runs/detect/ --name exp --save-crop
    #import torch
    #model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/yolov5/weights/best.pt')  # default
    #imgs = ['temp/gray_image.jpg'] 
    #results = model(imgs)
    #results.xyxy[0]  
    #results.pandas().xyxy[0]  
    #crops = results.crop(save=True)
    
    
    ##Check if directry present(True if object detection is correct)
    detect_folder='runs/detect/exp/crops/Letters/'
    isdir = os.path.isdir(detect_folder)
    if isdir == False:
        txts,score=paddle_man(img)   
    else:
        print('Dir exist')
        file = os.listdir(detect_folder)
        file=file[0]
        path = (detect_folder+""+file)
        src = cv2.imread(path)
        image = cv2.rotate(src, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
        cv2.imwrite("runs/temp/"+file, image)    
        #Convert to inverted imge
        gray_image_file='runs/temp/'+ file
        image_read=cv2.imread(gray_image_file)
        inverted_image = cv2.bitwise_not(image_read)
        cv2.imwrite("runs/temp/inverted.jpg", inverted_image)
        #Pass images to OCR
        txts,score=paddle_man("runs/temp/inverted.jpg")
        
    #torch.cuda.empty_cache()
    
    return txts,score

In [4]:
 import pandas as pd

In [5]:
def liveness_det_one(img_in):
    ##Livesness detection
    ##Datasouls
    import cv2
    %matplotlib inline
    from pylab import imshow
    import matplotlib.pyplot as plt
    import numpy as np
    import cv2
    import torch
    import seaborn as sns
    import pandas as pd
    import albumentations as albu
    from albumentations.pytorch.transforms import ToTensorV2
    from iglovikov_helper_functions.dl.pytorch.utils import tensor_from_rgb_image
    from iglovikov_helper_functions.utils.image_utils import load_rgb
    from datasouls_antispoof.pre_trained_models import create_model
    from datasouls_antispoof.class_mapping import class_mapping
    ###
    
    
    #model = create_model("tf_efficientnet_b3_ns")
    model1 = create_model("tf_efficientnet_b3_ns")
    model2 = create_model("swsl_resnext50_32x4d")
    model1.eval();
    model2.eval();
    image_replay = load_rgb(img)
    #imshow(image_replay)
    ###
    transform = albu.Compose([albu.PadIfNeeded(min_height=400, min_width=400),
                              albu.CenterCrop(height=400, width=400), 
                              albu.Normalize(p=1), 
                              albu.pytorch.ToTensorV2(p=1)], p=1)
    
    with torch.no_grad():
        prediction1 = model1(torch.unsqueeze(transform(image=image_replay)['image'], 0)).numpy()[0]
        prediction2 = model2(torch.unsqueeze(transform(image=image_replay)['image'], 0)).numpy()[0]

    df1 = pd.DataFrame({"prediction": prediction1, "class_name": class_mapping.keys()})
    df2 = pd.DataFrame({"prediction": prediction2, "class_name": class_mapping.keys()})
    #sns.barplot(data=df1, x="prediction", y="class_name") 
    #sns.barplot(data=df2, x="prediction", y="class_name") 
    
    res1_df=df1.loc[df1['prediction'].idxmax()]
    res2_df=df2.loc[df2['prediction'].idxmax()]
    redf1,redf2,pred1,pred2=res1_df.class_name,res2_df.class_name,res1_df.prediction,res2_df.prediction
    ###
    return redf1,redf2,pred1,pred2
    
    
    
    ##Livesness detection
    ##Neural Network(Binary classification)

def liveness_det_two(img_in):
    from imutils.video import VideoStream
    import tensorflow as tf
    import numpy as np
    import argparse
    import imutils
    import pickle
    import time
    import cv2
    import os
    
    liveness_model = tf.keras.models.load_model('LIVENESS_DETECT/face_liveness_detection/liveness_model_two.h5 ')
    le = pickle.loads(open('LIVENESS_DETECT/face_liveness_detection/label_encoder','rb').read())
    
    ##Read image and resize --
    frame = cv2.imread(img)
    face = cv2.resize(frame, (128,128))
    
    face = face.astype('float') / 255.0 
    face = tf.keras.preprocessing.image.img_to_array(face)
    # tf model require batch of data to feed in
    # so if we need only one image at a time, we have to add one more dimension
    # in this case it's the same with [face]
    face = np.expand_dims(face, axis=0)
    
    preds = liveness_model.predict(face)[0]
    j = np.argmax(preds)
    label = le.classes_[j]
    result2= 'fake' if label ==1 else 'real'
    res3df=pd.DataFrame(preds,columns=['dd'])
    tmp_df=res3df.loc[res3df['dd'].idxmax()]
    pred3=tmp_df.dd
    return result2,pred3


In [17]:
#Variables

folder='test/HERO_PRINTED/real/'
file='real7.jpeg'
img=(folder+""+file)
#display(img)
txts,score=main_code(img);
redf1,redf2,pred1,pred2=liveness_det_one(img);
liv3,pred3=liveness_det_two(img);


###Printing Results
print("\n")
print("Recognised Text :",txts)
print("Accuracy :",score)
print()
###Liveness Detection Results###
##Model1 & Model2
print("Image Liveness Detection Result:\n")
print("Model1 :",redf1," --> Accuracy:",pred1)
print("Model2 :",redf2," --> Accuracy:",pred2)

##Model3
print("Modle3 :",liv3," --> Accuracy:",pred3)

Dir exist
[2022/02/16 10:21:08] root WARNING: version PP-OCRv2 not support cls models, auto switch to version PP-OCR


detect: weights=['yolov5/yolov5/weights/large/best.pt'], source=temp/gray_image.jpg, imgsz=[416, 416], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect/, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.0-131-gc77a5a8 torch 1.10.0 CUDA:0 (GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model Summary: 367 layers, 46108278 parameters, 0 gradients, 107.8 GFLOPs
image 1/1 L:\CV\MAIN_PRO_POC\temp\gray_image.jpg: 416x256 1 Letters, Done. (0.021s)
Speed: 0.0ms pre-process, 21.0ms inference, 4.0ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs\detect\exp


Namespace(benchmark=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='paddleOCR_V2_EN_GEN_SRV_MOB_T/cls_model/ch_ppocr_mobile_v2.0_cls_train', cls_thresh=0.9, cpu_threads=10, det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='paddleOCR_V2_EN_GEN_SRV_MOB_T/ch_PP-OCRv2_det_distill_train', det_pse_box_thresh=0.85, det_pse_box_type='box', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_polygon=True, e2e_pgnet_score_thresh=0.5, e2e_pgnet_valid_set='totaltext', enable_mkldnn=False, gpu_mem=500, hel

In [11]:
##Liveness DetectOOnly###
##Liveness DetectOOnly###

#Variables

folder='test/HERO_PRINTED/real/'
file='real1.jpeg'
img=(folder+""+file)
#display(img)
redf1,redf2,pred1,pred2=liveness_det_one(img);
liv3,pred3=liveness_det_two(img);

###Liveness Detection Results###
##Model1 & Model2
print("Image Liveness Detection Result:\n")
print("Model1 :",redf1," --> Accuracy:",pred1)
print("Model2 :",redf2," --> Accuracy:",pred2)

##Model3
print("Modle3 :",liv3," --> Accuracy:",pred3)

Image Liveness Detection Result:

Model1 : real  --> Accuracy: 0.5863823
Model2 : replay  --> Accuracy: 0.63051635
Modle3 : fake  --> Accuracy: 0.9946108


In [ ]:
Models:
    1.YOLO V5 - Object detection(ROI)  -->Custom Dataset(1000 Images)
    2.OCR Mode - Text recognition      -->Pre trained
    
    3.Liveness Detection - Model1 (Replay/printed) -->Pre trained
    4.Liveness Detection - Model2 (Replay/printed) -->Pre trained
    5.Liveness Detection - Model3 (Replay)         -->Custom Dataset(200 replay Images)

    
Liveness Detection(Spoofing methods):
    1.Replay  (Images captured from computer/tab/phone screens)
    2.Printed (Images captured from printed papers)
    
Dataset: 
    1.replay  - 2000+ Images
    2.printed - 2000+ images

------------------------------------------------
    
OCR Custom Model for VIN Detection:
    1.Labelling: 
    2.Training : 
    
  


In [ ]:
Requirements:
    VIN recognition
    Replay image detection
    printed image dettection

In [19]:
    from paddleocr import PaddleOCR,draw_ocr
    from PIL import ImageFont
    import PIL
    from PIL import Image
    import cv2
    from matplotlib import pyplot as plt
    %matplotlib inline
    import os
    import shutil
    import pandas as pd
    from pylab import imshow
    import numpy as np
    import torch
    import seaborn as sns
    import albumentations as albu
    from albumentations.pytorch.transforms import ToTensorV2
    from iglovikov_helper_functions.dl.pytorch.utils import tensor_from_rgb_image
    from iglovikov_helper_functions.utils.image_utils import load_rgb
    from datasouls_antispoof.pre_trained_models import create_model
    from datasouls_antispoof.class_mapping import class_mapping
    from imutils.video import VideoStream
    import tensorflow as tf
    import argparse
    import imutils
    import pickle
    import time
    